In [1]:
import pandas as pd
from datasets import load_from_disk
import json

/data/ephemeral/home/miniconda3/envs/s1_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
 # JSON 파일 경로
file_path = '../outputs/train_dataset/predictions.json'

# JSON 파일 열기 및 데이터 로드
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터를 Pandas 데이터프레임으로 변환
df = pd.DataFrame.from_dict(data, orient='index')

df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df.head()

,id,0
0,mrc-0-003264,한보철강
1,mrc-0-004762,1871년
2,mrc-1-001810,나뭇잎
3,mrc-1-000219,금대야
4,mrc-1-000285,수평적 관계


In [29]:
dataset = load_from_disk("../data/raw/train_dataset/") 
validation_df = pd.DataFrame(dataset['validation'])

In [31]:
print(df.columns)
print(validation_df.columns)

Index(['id', 0], dtype='object')
Index(['title', 'context', 'question', 'id', 'answers', 'document_id',
       '__index_level_0__'],
      dtype='object')


In [32]:
# df의 'id'와 '0' 열, 그리고 validation_df의 'id'와 'answers' 열을 병합
merged_df = pd.merge(df, validation_df[['id', 'answers']], on='id', how='inner')

# df의 0 열과 validation_df의 answers['text'] 비교하여 일치 여부 확인
merged_df['is_answer_match'] = merged_df.apply(lambda row: row[0] == row['answers']['text'][0], axis=1)

# 결과 확인
print(merged_df[['id', 0, 'answers', 'is_answer_match']].head())

# 일치 여부 통계 확인
match_counts = merged_df['is_answer_match'].value_counts(normalize=True)
print(f"일치하는 비율: {match_counts[True]:.2%}")
print(f"일치하지 않는 비율: {match_counts[False]:.2%}")


             id       0                                      answers  \
0  mrc-0-003264    한보철강    {'answer_start': [284], 'text': ['한보철강']}   
1  mrc-0-004762   1871년   {'answer_start': [146], 'text': ['1871년']}   
2  mrc-1-001810     나뭇잎     {'answer_start': [517], 'text': ['나뭇잎']}   
3  mrc-1-000219     금대야    {'answer_start': [1109], 'text': ['금대야']}   
4  mrc-1-000285  수평적 관계  {'answer_start': [386], 'text': ['수평적 관계']}   

   is_answer_match  
0             True  
1             True  
2             True  
3             True  
4             True  
일치하는 비율: 48.33%
일치하지 않는 비율: 51.67%


In [52]:
# 행과 열 출력 설정을 기본값으로 되돌리기
pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
pd.set_option('display.max_rows', None)  # 모든 행 출력
# pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.colheader_justify', 'left')



In [55]:
# df의 'id'와 '0' 열, 그리고 validation_df의 'id'와 'answers' 열을 병합
merged_df = pd.merge(df, validation_df[['id', 'answers']], on='id', how='inner')

# df의 0 열과 validation_df의 answers['text'][0] 비교하여 일치 여부 확인
merged_df['is_answer_match'] = merged_df.apply(lambda row: row[0] == row['answers']['text'][0], axis=1)

# 일치하지 않는 경우만 필터링
mismatched_answers = merged_df[merged_df['is_answer_match'] == False]

# answers['text'][0]만 추출해서 새로운 열에 저장
mismatched_answers['validation_answer'] = mismatched_answers['answers'].apply(lambda x: x['text'][0])

# 일치하지 않는 경우의 id, df의 답변(0 열), validation_df의 답변(validation_answer) 출력
print(mismatched_answers[['id', 0, 'validation_answer']].to_string(index=False))



id           0                                                              validation_answer                                               
mrc-0-005106                                                    쇼와 천황의 옥음방송                                                             옥음방송
mrc-0-002875                                                            프랑스                                                              스페인
mrc-0-003828                                                        1729년까지                                                           20세기 초
mrc-0-003032                                                   1967년 8월 16일                                                    1967년 11월 15일
mrc-1-000724                                                          1969년                                                            1965년
mrc-0-002138                                                            톨루이                                                           아리크 부케
mrc-0-003115 

/tmp/ipykernel_315099/363302542.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_answers['validation_answer'] = mismatched_answers['answers'].apply(lambda x: x['text'][0])


In [54]:
print(mismatched_answers[['id', 0, 'validation_answer']].to_string(index=False))


id           0                                                              validation_answer                                               
mrc-0-005106                                                    쇼와 천황의 옥음방송                                                             옥음방송
mrc-0-002875                                                            프랑스                                                              스페인
mrc-0-003828                                                        1729년까지                                                           20세기 초
mrc-0-003032                                                   1967년 8월 16일                                                    1967년 11월 15일
mrc-1-000724                                                          1969년                                                            1965년
mrc-0-002138                                                            톨루이                                                           아리크 부케
mrc-0-003115 